## 使用 ChatGLM 提供对话效果

In [ ]:
!pip install transformers
!pip install icetk
!pip install cpm_kernels

In [ ]:

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

print(locale.getpreferredencoding())

!pip install --upgrade protobuf

In [ ]:

question = """
自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。

根据以上信息，请回答下面的问题：

Q: 你们的退货政策是怎么样的？
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

In [ ]:

question = """
Q: 你们的退货政策是怎么样的？
A: 
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

In [ ]:

question = """
我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆.

根据以上信息，请回答下面的问题：

Q: 你们能配送到三亚吗？
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

In [ ]:

question = """
我们支持全国大部分省份的配送，包括北京、上海、天津、重庆、河北、山西、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、广东、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏和新疆.但是不能配送到东三省

根据以上信息，请回答下面的问题：

Q: 你们能配送到哈尔滨吗？
"""
response, history = model.chat(tokenizer, question, history=[])
print(response)

## 将 ChatGLM 封装成 LLM

In [ ]:
!pip install openai
!pip install faiss-gpu
!pip install llama_index
!pip install langchain

In [ ]:

import openai, os
import faiss
from llama_index import SimpleDirectoryReader, LangchainEmbedding, ServiceContext, GPTVectorStoreIndex, StorageContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from llama_index.node_parser import SimpleNodeParser

from langchain.llms.base import LLM
from llama_index import LLMPredictor
from typing import Optional, List, Mapping, Any

class CustomLLM(LLM):
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response, history = model.chat(tokenizer, prompt, history=[])
        return response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "chatglm-6b-int4"}

    @property
    def _llm_type(self) -> str:
        return "custom"

In [ ]:
!pip install sentence_transformers

In [ ]:

from langchain.text_splitter import SpacyTextSplitter
from llama_index.vector_stores.faiss import FaissVectorStore

llm_predictor = LLMPredictor(llm=CustomLLM())

text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=20)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./drive/MyDrive/colab_data/faq/').load_data()
nodes = parser.get_nodes_from_documents(documents)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:

from llama_index import QuestionAnswerPrompt

QA_PROMPT_TMPL = (
    "{context_str}"
    "\n\n"
    "根据以上信息，请回答下面的问题：\n"
    "Q: {query_str}\n"
    )
QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)

query_engine = index.as_query_engine(text_qa_template=QA_PROMPT, verbose=True)
response = query_engine.query("请问你们海南能发货吗？")
print(response)


## 开源模型的不足之处

In [ ]:
!pip install spacy
!python -m spacy download zh_core_web_sm

In [ ]:

text_splitter = SpacyTextSplitter(pipeline="zh_core_web_sm", chunk_size = 128, chunk_overlap=32)
parser = SimpleNodeParser(text_splitter=text_splitter)
documents = SimpleDirectoryReader('./drive/MyDrive/colab_data/zhaohuaxishi/').load_data()
nodes = parser.get_nodes_from_documents(documents)

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
))
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm_predictor=llm_predictor)

dimension = 768
faiss_index = faiss.IndexFlatIP(dimension)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:

# query will use the same embed_model
from llama_index import QuestionAnswerPrompt

# openai.api_key = os.environ.get("OPENAI_API_KEY")

QA_PROMPT_TMPL = (
    "下面的内容来自鲁迅先生的散文集《朝花夕拾》，很多内容是以第一人称写的 \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "根据这些信息，请回答问题: {query_str}\n"
    "如果您不知道的话，请回答不知道\n"
)
QA_PROMPT = QuestionAnswerPrompt(QA_PROMPT_TMPL)

query_engine = index.as_query_engine(similarity_top_k = 1, text_qa_template=QA_PROMPT, verbose=True)
response = query_engine.query("鲁迅先生在日本学习医学的老师是谁？")
print(response)


In [ ]:
response = query_engine.query("鲁迅先生是在日本的哪个城市学习医学的？")
print(response)

In [ ]:

question = """Consideration proudct : 工厂现货PVC充气青蛙夜市地摊热卖充气玩具发光蛙儿童水上玩具

1. Compose human readale product title used on Amazon in english within 20 words.
2. Write 5 selling points for the products in Amazon.
3. Evaluate a price range for this product in U.S.

Output the result in json format with three properties called title, selling_points and price_range"""
response, history = model.chat(tokenizer, question, history=[])
print(response)